In [ ]:
# Library preparation
import cv2
import numpy as np
import matplotlib.pyplot as pl

In [ ]:
# loading samples
img1 = cv2.imread('')



In [ ]:
# Transfering samples to gray colors
def image_to_gray(img):
    """
    Transfer given image to gray

    Parameters:
    img (ndarray): Image

    Returns: ndarray: Image converted to gray scale
    """
    return cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

